In [3]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import csv
import sys
from io import StringIO


original_stdout = sys.stdout

sys.stdout = StringIO()

np.random.seed(42)
GAP_THRESHOLD = 0.15
N_REPEATS = 10
SCORING_METRIC = 'r2'


pattern = []
with open('4-pattern1.csv', 'r', encoding='utf-8-sig') as fhd:
    fhd_csv = csv.reader(fhd)
    for line in fhd_csv:
        pattern.append(line)
pattern = np.array(pattern, dtype='float64')


pattern = np.where(np.isinf(pattern), np.nan, pattern)
pattern = np.nan_to_num(pattern, nan=np.nanmean(pattern) if not np.isnan(np.nanmean(pattern)) else 0)

min_vals = np.min(pattern, axis=0)
max_vals = np.max(pattern, axis=0)
range_vals = np.where(max_vals - min_vals == 0, 1, max_vals - min_vals)
pattern_normalized = (pattern - min_vals) / range_vals

scaler = StandardScaler()
pattern_scaled = scaler.fit_transform(pattern_normalized)


label_data = []
with open('output-IMS10.csv', 'r', encoding='utf-8-sig') as fhl:
    fhl_csv = csv.reader(fhl)
    for line in fhl_csv:
        label_data.append(line)
label_data = np.array(label_data, dtype='float64')

groups = label_data[:, 0]
label_c = label_data[:, 1]
label_c = np.exp(label_c)


unique_groups = np.unique(groups)
train_groups, test_groups = train_test_split(
    unique_groups,
    test_size=99/491,
    random_state=42
)
train_mask = np.isin(groups, train_groups)
test_mask = np.isin(groups, test_groups)

X_train_raw = pattern_scaled[train_mask]
y_train = label_c[train_mask]
X_test_raw = pattern_scaled[test_mask]
y_test = label_c[test_mask]


pca = PCA(n_components=12)
X_train_pca_all = pca.fit_transform(X_train_raw)
X_test_pca_all = pca.transform(X_test_raw)

X_train_pca = X_train_pca_all[:, :6]
X_test_pca = X_test_pca_all[:, :6]


def mean_relative_error(y_true, y_pred):
    mask = y_true != 0
    if np.sum(mask) == 0:
        return 0.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mre = mean_relative_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'mre': mre,
        'r2': r2
    }


def permutation_feature_importance(model, X, y, n_repeats=10, scoring='r2'):
    y_pred = model.predict(X)
    if scoring == 'r2':
        baseline_score = r2_score(y, y_pred)
    elif scoring == 'mae':
        baseline_score = -mean_absolute_error(y, y_pred)
    elif scoring == 'mse':
        baseline_score = -mean_squared_error(y, y_pred)
    else:
        raise ValueError(f"Unsupported scoring metric: {scoring}")
    
    n_features = X.shape[1]
    importances = np.zeros((n_repeats, n_features))
    
    for i in range(n_repeats):
        for j in range(n_features):
            X_permuted = X.copy()
            np.random.shuffle(X_permuted[:, j])
            
            y_pred_permuted = model.predict(X_permuted)
            if scoring == 'r2':
                score = r2_score(y, y_pred_permuted)
            elif scoring == 'mae':
                score = -mean_absolute_error(y, y_pred_permuted)
            elif scoring == 'mse':
                score = -mean_squared_error(y, y_pred_permuted)
            
            importances[i, j] = baseline_score - score
    
    mean_importances = np.mean(importances, axis=0)
    std_importances = np.std(importances, axis=0)
    
    return mean_importances, std_importances, baseline_score


best_params = {
    'n_estimators': 50,
    'max_depth': 20,
    'min_samples_split': 10,
    'min_samples_leaf': 2,
    'max_features': 'sqrt'
}

model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_pca, y_train, 
    test_size=0.2, 
    random_state=42
)

model.fit(X_tr, y_tr)


y_tr_pred = model.predict(X_tr)
y_val_pred = model.predict(X_val)
y_train_pred = model.predict(X_train_pca)
y_test_pred = model.predict(X_test_pca)

tr_metrics = calculate_metrics(y_tr, y_tr_pred)
val_metrics = calculate_metrics(y_val, y_val_pred)
train_metrics = calculate_metrics(y_train, y_train_pred)
test_metrics = calculate_metrics(y_test, y_test_pred)

train_val_gap = abs(train_metrics['r2'] - val_metrics['r2'])
train_test_gap = abs(train_metrics['r2'] - test_metrics['r2'])

is_valid = (train_val_gap < GAP_THRESHOLD) and (train_test_gap < GAP_THRESHOLD)

# Final model training
final_model = RandomForestRegressor(**best_params, bootstrap=True, random_state=42, n_jobs=-1)
final_model.fit(X_train_pca, y_train)
final_y_train_pred = final_model.predict(X_train_pca)
final_y_test_pred = final_model.predict(X_test_pca)

sys.stdout = original_stdout


print("\n===================== Permutation Feature Importance Analysis =====================")
importances, importances_std, baseline_score = permutation_feature_importance(
    final_model, X_test_pca, y_test, 
    n_repeats=N_REPEATS, 
    scoring=SCORING_METRIC
)

feature_names = [f'PCA Component {i+1}' for i in range(len(importances))]

print(f"Baseline {SCORING_METRIC} score: {baseline_score:.6f}")
print("\nFeature Importance (Higher value = more important):")
for name, imp, std in zip(feature_names, importances, importances_std):
    print(f"{name}: Mean Importance = {imp:.6f} (Standard Deviation = {std:.6f})")

pca_pred = np.full(len(pattern_scaled), np.nan)
pca_pred[train_mask] = final_y_train_pred
pca_pred[test_mask] = final_y_test_pred

results_df = pd.DataFrame({
    "Original_Index": range(1, len(pattern_scaled)+1),
    "PCA_Dimensions": 6,
    "Group": groups,
    "True_Label": label_c,
    "Dataset": np.where(np.isin(groups, train_groups), "Training Set", "Test Set"),
    "Predicted_Label": pca_pred
})

summary_df = pd.DataFrame({
    "PCA_Components": [6],
    "Best_Params": [str(best_params)],
    "Train_R2": [round(train_metrics['r2'], 6)],
    "Train_MAE": [round(train_metrics['mae'], 6)],
    "Train_MSE": [round(train_metrics['mse'], 6)],
    "Train_MRE": [round(train_metrics['mre'], 6)],
    "Train_RMSE": [round(train_metrics['rmse'], 6)],
    "Val_R2": [round(val_metrics['r2'], 6)],
    "Val_MAE": [round(val_metrics['mae'], 6)],
    "Val_MSE": [round(val_metrics['mse'], 6)],
    "Val_MRE": [round(val_metrics['mre'], 6)],
    "Val_RMSE": [round(val_metrics['rmse'], 6)],
    "Test_R2": [round(test_metrics['r2'], 6)],
    "Test_MAE": [round(test_metrics['mae'], 6)],
    "Test_MSE": [round(test_metrics['mse'], 6)],
    "Test_MRE": [round(test_metrics['mre'], 6)],
    "Test_RMSE": [round(test_metrics['rmse'], 6)],
    "Train_Val_Gap": [round(train_val_gap, 6)],
    "Train_Test_Gap": [round(train_test_gap, 6)],
    "Is_Valid": [is_valid]
})

importance_df = pd.DataFrame({
    "Feature_Name": feature_names,
    "Mean_Importance": [round(imp, 6) for imp in importances],
    "Importance_Std": [round(std, 6) for std in importances_std],
    "Baseline_Score": [round(baseline_score, 6)] * len(importances),
    "Scoring_Metric": [SCORING_METRIC] * len(importances),
    "Permutation_Repeats": [N_REPEATS] * len(importances)
})

with pd.ExcelWriter('RF_PCA_6D_Results-IMS-Permutation_Feature_Importance.xlsx', engine='openpyxl') as writer:
    summary_df.to_excel(writer, sheet_name='PCA_Performance_Summary', index=False)
    results_df.to_excel(writer, sheet_name='Full_Dimension_Predictions', index=False)
    importance_df.to_excel(writer, sheet_name='Permutation_Feature_Importance', index=False)

print(f"\n📁 Results exported to: RF_PCA_6D_Results-IMS-Permutation_Feature_Importance.xlsx")


===================== Permutation Feature Importance Analysis =====================
Baseline r2 score: 0.741859

Feature Importance (Higher value = more important):
PCA Component 1: Mean Importance = 0.141933 (Standard Deviation = 0.040137)
PCA Component 2: Mean Importance = 0.780052 (Standard Deviation = 0.079832)
PCA Component 3: Mean Importance = 0.093063 (Standard Deviation = 0.017259)
PCA Component 4: Mean Importance = 0.007797 (Standard Deviation = 0.007750)
PCA Component 5: Mean Importance = 0.015848 (Standard Deviation = 0.004200)
PCA Component 6: Mean Importance = 0.012564 (Standard Deviation = 0.003577)

📁 Results exported to: RF_PCA_6D_Results-IMS-Permutation_Feature_Importance.xlsx
